## Testing Nasmyth2 Rotator

Craig Lage - 29 Jun 22


In [ ]:
import sys, time, os

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import logging

from lsst.ts import salobj

from lsst.ts.observatory.control.auxtel.atcs import ATCS
from lsst.ts.observatory.control.auxtel.latiss import LATISS
from lsst.ts.observatory.control.utils import RotType

from lsst.ts.observing.utilities.decorated_logger import DecoratedLogger

from astropy.time import Time, TimeDelta
from lsst_efd_client import EfdClient

In [ ]:
logger = DecoratedLogger.get_decorated_logger()
logger.level = logging.DEBUG

In [ ]:
logger.info(os.environ["OSPL_URI"])
logger.info(os.environ["LSST_DDS_PARTITION_PREFIX"])

### Instantiate the control classes

In [ ]:
domain = salobj.Domain()
atcs = ATCS(domain)
#latiss = LATISS(domain)
await asyncio.gather(atcs.start_task)#, latiss.start_task)

In [ ]:
await atcs.enable()

In [ ]:
current_az = 0.0
current_el = 80.0
current_rot = 0.0
await atcs.point_azel(current_az, current_el, rot_tel=current_rot)

In [ ]:
current_az = 180.0
current_el = 80.0
current_rot = -160.0
await atcs.point_azel(current_az, current_el, rot_tel=current_rot)

In [ ]:
current_az = 180.0
current_el = 80.0
current_rot = 160.0
await atcs.point_azel(current_az, current_el, rot_tel=current_rot)

In [ ]:
current_az = 180.0
current_el = 80.0
current_rot = -160.0
await atcs.point_azel(current_az, current_el, rot_tel=current_rot)

In [ ]:
current_az = 180.0
current_el = 80.0
current_rot = 0.0
await atcs.point_azel(current_az, current_el, rot_tel=current_rot)

In [ ]:
current_az = 180.0
current_el = 80.0
current_rot = 80.0
await atcs.point_azel(current_az, current_el, rot_tel=current_rot)

Now we'll analyze the data.

We'll access the EFD instance deployed at the Summit.

In [ ]:
client = EfdClient('summit_efd')

In [ ]:
t_end = Time(time.time(),format='unix', scale='utc')
#t_end = Time("2021-05-04T22:04:00", scale='utc')   # timestamp at end of desired data 
nsec = 0.21*3600 # how many seconds of data to retrieve
torque = await client.select_packed_time_series("lsst.sal.ATMCS.measuredTorque", 'nasmyth2MotorTorque',
                                              t_end - TimeDelta(nsec, format='sec'), t_end)
angle = await client.select_packed_time_series("lsst.sal.ATMCS.mount_Nasmyth_Encoders", 'nasmyth2CalculatedAngle',
                                              t_end - TimeDelta(nsec, format='sec'), t_end)



In [ ]:
ax1 = torque['nasmyth2MotorTorque'].plot(figsize=(16,8), title='Nasmyth motor torque', legend=True)
ax1.axhline(3.0, color="blue", linestyle="--")
ax1.axhline(-3.0, color="blue", linestyle="--")
plt.ylim(-3.5,3.5)
angle['nasmyth2CalculatedAngle'].plot(legend=True, secondary_y=True)
plt.savefig("/home/craiglagegit/DATA/Nasmyth_Torque_12Jul22.pdf")

In [ ]:
torqueList = torque.values.tolist()
angleList = angle.values.tolist()
plt.figure()
plt.plot(np.array(angleList)[:,0],np.array(torqueList)[:,0])
plt.plot([-160,160],[3.0,3.0], color='red', ls='--')
plt.plot([-160,160],[-3.0,-3.0], color='red', ls='--')
plt.arrow(-140, 2.5, 50,0, width=0.1,head_length = 5.0, color='green')
plt.arrow(140, -2.5, -50,0, width=0.1,head_length = 5.0, color='green')
plt.xlabel("Rotator angle(degrees)")
plt.ylabel("Torque (amps)")
plt.savefig("/home/craiglagegit/DATA/Torque_vs_Angle_12Jul22.pdf")